In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero
dir_ = '../../../../data'
group_dir_ = os.path.join(dir_, 'groups/random')

In [2]:
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'
df = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

num_user = len(svd['uid'].unique())
num_user

953

In [3]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [4]:
pop_dict = {}
for i in df['tid'].unique():
    pop_dict[i] = len(df[df['tid']==i])/num_user

In [5]:
svd['count'] = 1
svd[:5]

,uid,tid,rating,count
0,0,9,2.243909,1
1,0,15,2.160610,1
2,0,22,2.145235,1
3,0,28,2.204445,1
4,0,33,2.166677,1


In [6]:
# Count POP to DF
tid_list = []
pop_list = []
for i in df['tid'].unique():
    tid_list.append(i)
    pop_list.append(len(df[df['tid']==i])/num_user*5)
    
d = {'tid': tid_list, 'rating': pop_list}
df_pop = pd.DataFrame(data=d)
df_pop = df_pop.sort_values(by=['rating'], ascending=False)
df_pop[:10]

,tid,rating
70,70,4.034627
5521,5521,2.911857
390,390,2.822665
2519,2519,2.801679
83,83,2.796432
13496,13496,2.785939
210,210,2.686254
5716,5716,2.681007
1036,1036,2.544596
1464,1464,2.528856


In [7]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [8]:
NOVs = []
COVs = []
HitNOVs = []
HitCOVs = []
for lambda_ in tqdm(lambdas):
    top_n_size = 50
    top_n_lists_g = []
    top_n_items_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []
        top_n_items = pd.DataFrame()

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
            
            if len(top_n_items) == 0:
                top_n_items = rating_table
            else:
                top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()
                
        top_n_lists_g.append(top_n_lists)   
        top_n_items_g.append(top_n_items)
        
    COV = []    
    NOV = []
    HitNOV = []
    HitCOV = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]
        top_n_items = top_n_items_g[i]

        nov = 0
        hitNov = 0
        hits = np.zeros(len(df['tid'].unique()), dtype=bool)
        for j in (range(len(groups))):
            group = groups[j]
            top_n_list = top_n_lists[j]
            
            test_g = pd.DataFrame()
            for uid in group:
                t = test[test['uid'] == uid]
                if len(test_g) == 0:
                    test_g = t
                else:
                    test_g = test_g.set_index('tid').add(t.set_index('tid'), fill_value=0).reset_index()
            test_g['rating'] /= len(group)
            
            for k in range(top_n_size):
                tid = top_n_list[k]
                t = test_g[test_g['tid'] == tid]
                nov += (1-pop_dict[tid])
                if len(t) > 0:
                    hits[int(tid)] = True
                    hitNov += (1-pop_dict[tid])
        NOV.append(nov/top_n_size/len(groups))
        COV.append(len(top_n_items)/len(df['tid'].unique()))
        HitNOV.append(hitNov/top_n_size/len(groups))
        HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
    NOVs.append(NOV)
    COVs.append(COV)
    HitNOVs.append(HitNOV)
    HitCOVs.append(HitCOV)
    
for NOV in NOVs:
    print(NOV)
print()
for HitNOV in HitNOVs:
    print(HitNOV)
print()  
for COV in COVs:
    print(COV)
print()    
for HitCOV in HitCOVs:
    print(HitCOV)


[0.5304512067156409, 0.5304512067156492, 0.5304512067156464, 0.5304512067156426, 0.5304512067156388, 0.530451206715638, 0.5304512067156372, 0.5304512067156364, 0.5304512067156357]
[0.53182334423802, 0.5314128718541338, 0.5311274436322415, 0.530975534323751, 0.530861768964101, 0.5308122955373151, 0.5307206786177239, 0.5307040423724592, 0.530648920307064]
[0.5381587115433852, 0.5357961079241925, 0.534522031267928, 0.533867012757506, 0.533292069015901, 0.5327262206036678, 0.5325750615041405, 0.5323475740768504, 0.5320585408957861]
[0.5504315430264478, 0.5456041522537368, 0.5427388080101043, 0.5404800353454543, 0.5393224593887407, 0.5381664712054818, 0.5371216944280328, 0.5365272572827663, 0.5361539736013692]
[0.5686604001516624, 0.5625530534490297, 0.5582965778126731, 0.5548538134423233, 0.5525524991034328, 0.5494648478488987, 0.5474727309601707, 0.5460508669364919, 0.5451635279173804]
[0.584653107832847, 0.5868775674360672, 0.5830430220356911, 0.5809426188766864, 0.5773504057805484, 0.5